# Hull Tactical - V5 FULL PROMETHEUS

**This implements ALL 30 breakthrough insights:**
- Phase Transition Detection (Variance Compression, Critical Slowing Down, Anomalous Dimension)
- Ensemble Intelligence (Dempster-Shafer Fusion, Value Clustering, CIC Confidence)
- Adaptive Position Sizing (Kelly + Regime + Risk Signals)
- Feature Engineering (Temperature, Order Parameters, Cross-Domain Correlation)

**Target**: Break 0.244 → Achieve 1.5+ Sharpe

In [ ]:
import os
import pickle
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import RobustScaler
from scipy.signal import savgol_filter

warnings.filterwarnings('ignore')
np.random.seed(42)

# Paths
if os.path.exists('/kaggle/input/hull-tactical-market-prediction'):
    DATA_DIR = Path('/kaggle/input/hull-tactical-market-prediction')
    # Try to load pre-trained artifacts
    for name in ['hull-artifacts-v5', 'hull-artifacts-v4', 'hull-artifacts']:
        test = Path(f'/kaggle/input/{name}')
        if test.exists():
            ARTIFACTS_DIR = test
            break
    else:
        ARTIFACTS_DIR = Path('/kaggle/working')
else:
    DATA_DIR = Path('/home/user/aimo3/hull/hull-tactical-market-prediction')
    ARTIFACTS_DIR = Path('/home/user/aimo3/hull/artifacts_v5')

ARTIFACTS_DIR.mkdir(exist_ok=True, parents=True)
print(f"Data: {DATA_DIR}")
print(f"Artifacts: {ARTIFACTS_DIR}")

In [ ]:
# =============================================================================
# V5 CONFIG - FULL PROMETHEUS PARAMETERS
# =============================================================================
CONFIG = {
    # Position Sizing - ADAPTIVE
    'base_position': 1.0,
    'risk_aversion': 30.0,       # Even lower - trust the model more
    'scale_factor': 150.0,       # Higher responsiveness
    'min_position': 0.0,         # Can go flat
    'max_position': 2.0,         # Full leverage
    
    # Risk Adjustment Factors
    'var_compression_factor': 0.6,    # Reduce when variance compresses
    'ac1_rising_factor': 0.7,         # Reduce when AC1 rising
    'correlation_surge_factor': 0.5,  # Reduce when correlations surge
    'high_conflict_factor': 0.6,      # Reduce when models disagree
    'vol_expanding_factor': 0.8,      # Reduce when volatility expanding
    
    # Model Parameters
    'lgb_params': {
        'objective': 'regression',
        'metric': 'mse',
        'boosting_type': 'gbdt',
        'num_leaves': 127,        # Deeper
        'max_depth': 10,
        'learning_rate': 0.015,
        'feature_fraction': 0.6,
        'bagging_fraction': 0.6,
        'bagging_freq': 3,
        'min_child_samples': 30,
        'lambda_l1': 0.05,
        'lambda_l2': 0.05,
        'verbose': -1,
        'seed': 42,
        'n_jobs': -1
    },
    'xgb_params': {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': 8,
        'learning_rate': 0.015,
        'subsample': 0.6,
        'colsample_bytree': 0.6,
        'min_child_weight': 30,
        'reg_alpha': 0.05,
        'reg_lambda': 0.05,
        'seed': 42,
        'n_jobs': -1,
        'verbosity': 0
    },
    'n_models': 7,               # More models for better ensemble
    'num_boost_round': 2000,
    'early_stopping': 150
}

print("V5 FULL PROMETHEUS Config:")
print(f"  Position range: [{CONFIG['min_position']}, {CONFIG['max_position']}]")
print(f"  Risk aversion: {CONFIG['risk_aversion']}")
print(f"  Scale factor: {CONFIG['scale_factor']}")
print(f"  Models: {CONFIG['n_models']} LGB + {CONFIG['n_models']} XGB")

In [ ]:
# =============================================================================
# INSIGHT #1-4: PHASE TRANSITION DETECTION
# =============================================================================

def compute_autocorrelation(x, tau):
    """Autocorrelation at lag tau."""
    if tau >= len(x) or len(x) < 10:
        return 0.0
    x_centered = x - np.mean(x)
    n = len(x) - tau
    if n < 5:
        return 0.0
    numer = np.sum(x_centered[:n] * x_centered[tau:tau+n])
    denom = np.sum(x_centered ** 2)
    return numer / (denom + 1e-10)


def compute_anomalous_dimension(x, tau, K=1.0):
    """Δ(t,τ) = [K - ln|G(t,τ)|] / ln(τ) - STRONGEST crash predictor"""
    if tau <= 1 or len(x) < tau + 10:
        return 0.0
    G = compute_autocorrelation(x, tau)
    if abs(G) < 1e-10:
        G = 1e-10
    return (K - np.log(abs(G))) / np.log(tau)


def compute_multiscale_delta(x, tau_min=5, tau_max=30):
    """Average anomalous dimension across scales."""
    if len(x) < tau_max + 10:
        return 0.0
    taus = range(tau_min, min(tau_max, len(x) // 3))
    deltas = [compute_anomalous_dimension(x, tau) for tau in taus]
    deltas = [d for d in deltas if not np.isnan(d) and not np.isinf(d) and abs(d) < 100]
    return np.median(deltas) if deltas else 0.0


def detect_variance_compression(returns, window_short=21, window_long=63):
    """Insight #1: Variance compression precedes regime shifts"""
    if len(returns) < window_long:
        return 0.0, 1.0
    recent_var = returns[-window_short:].var()
    baseline_var = returns[-window_long:-window_short].var()
    ratio = recent_var / (baseline_var + 1e-10)
    compression = ratio < 0.5  # Compression detected
    return float(compression), ratio


def detect_critical_slowing_down(returns, window=63):
    """Insight #2: Rising AC1 = approaching transition"""
    if len(returns) < window:
        return 0.0, 0.0
    recent = returns[-window:]
    try:
        ac1 = np.corrcoef(recent[:-1], recent[1:])[0, 1]
    except:
        ac1 = 0.0
    ac1_rising = ac1 > 0.5  # High autocorrelation
    return float(ac1_rising), ac1

print("Phase transition detection functions defined.")

In [ ]:
# =============================================================================
# INSIGHT #5-8: ENSEMBLE INTELLIGENCE
# =============================================================================

def dempster_shafer_fusion(predictions, reliabilities=None):
    """Insight #5: Reliability-weighted belief fusion with conflict detection"""
    if reliabilities is None:
        reliabilities = np.ones(len(predictions))
    reliabilities = np.array(reliabilities)
    predictions = np.array(predictions)
    
    # Normalize reliabilities
    r_sum = reliabilities.sum()
    if r_sum < 1e-10:
        return np.mean(predictions), 0.0, 1.0
    weights = reliabilities / r_sum
    
    # Fused prediction
    fused = np.sum(predictions * weights)
    
    # Compute conflict (direction disagreement)
    n = len(predictions)
    conflict = 0.0
    pairs = 0
    for i in range(n):
        for j in range(i+1, n):
            sign_disagree = float(np.sign(predictions[i]) != np.sign(predictions[j]))
            conflict += reliabilities[i] * reliabilities[j] * sign_disagree
            pairs += 1
    if pairs > 0:
        conflict /= pairs
    
    confidence = 1 - conflict
    return fused, confidence, conflict


def value_clustering(predictions, threshold=0.05):
    """Insight #6: Cluster predictions by relative proximity (92% error reduction)"""
    predictions = list(predictions)
    if len(predictions) < 2:
        return predictions[0] if predictions else 0.0, 1.0
    
    def rel_dist(a, b):
        return abs(a - b) / (max(abs(a), abs(b)) + 1e-8)
    
    clusters = []
    for pred in predictions:
        added = False
        for cluster in clusters:
            if rel_dist(pred, np.median(cluster)) < threshold:
                cluster.append(pred)
                added = True
                break
        if not added:
            clusters.append([pred])
    
    largest = max(clusters, key=len)
    return np.median(largest), len(largest) / len(predictions)


def compute_cic_functional(predictions, lambda_=0.3, gamma=0.1):
    """Insight #17: CIC confidence: F = Φ - λH + γC"""
    predictions = np.array(predictions)
    if len(predictions) < 2:
        return 0.5
    
    mean_pred = np.mean(predictions)
    std_pred = np.std(predictions)
    
    # Φ: Integrated information (coherence)
    phi = 1 - std_pred / (abs(mean_pred) + 1e-8)
    phi = np.clip(phi, 0, 1)
    
    # H: Entropy (uncertainty)
    h = std_pred / (std_pred + 1)
    
    # C: Causal power (coefficient of variation inverse)
    cv = std_pred / (abs(mean_pred) + 1e-8)
    c = 1 / (1 + cv)
    
    F = phi - lambda_ * h + gamma * c
    return np.clip(F, 0, 1)

print("Ensemble intelligence functions defined.")

In [ ]:
# =============================================================================
# FULL PROMETHEUS FEATURE ENGINEERING
# =============================================================================

def add_full_prometheus_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add ALL PROMETHEUS features for training."""
    df = df.copy()
    
    # === BASIC ROLLING FEATURES ===
    key_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'M1', 'M2', 'M3', 'S1', 'S2', 'S3', 'E1', 'E2', 'P1', 'I1', 'I2']
    key_cols = [c for c in key_cols if c in df.columns]
    
    for col in key_cols:
        for window in [5, 10, 21, 63]:
            if len(df) >= window:
                df[f'{col}_ma{window}'] = df[col].rolling(window, min_periods=1).mean()
                df[f'{col}_std{window}'] = df[col].rolling(window, min_periods=1).std().fillna(0)
                # Z-score
                df[f'{col}_z{window}'] = (df[col] - df[f'{col}_ma{window}']) / (df[f'{col}_std{window}'] + 1e-8)
    
    # === LAGGED RETURNS ===
    if 'lagged_forward_returns' in df.columns:
        df['lagged_ret'] = df['lagged_forward_returns']
    elif 'forward_returns' in df.columns:
        df['lagged_ret'] = df['forward_returns'].shift(1).fillna(0)
    else:
        df['lagged_ret'] = 0
    
    for w in [5, 10, 21, 63, 126]:
        if len(df) >= w:
            df[f'ret_cumsum_{w}'] = df['lagged_ret'].rolling(w, min_periods=1).sum()
            df[f'ret_vol_{w}'] = df['lagged_ret'].rolling(w, min_periods=1).std().fillna(0)
            df[f'sharpe_{w}'] = df[f'ret_cumsum_{w}'] / (df[f'ret_vol_{w}'] * np.sqrt(w) + 1e-8)
            df[f'ret_skew_{w}'] = df['lagged_ret'].rolling(w, min_periods=w//2).skew().fillna(0)
            df[f'ret_kurt_{w}'] = df['lagged_ret'].rolling(w, min_periods=w//2).kurt().fillna(0)
    
    # === PHASE TRANSITION FEATURES ===
    
    # Variance Compression (Insight #1)
    if len(df) >= 63:
        df['var_21'] = df['lagged_ret'].rolling(21, min_periods=5).var().fillna(0)
        df['var_63'] = df['lagged_ret'].rolling(63, min_periods=10).var().fillna(0)
        df['var_ratio'] = df['var_21'] / (df['var_63'] + 1e-10)
        df['var_compression'] = (df['var_ratio'] < 0.5).astype(float)
        df['var_expansion'] = (df['var_ratio'] > 2.0).astype(float)
    else:
        df['var_21'] = df['var_63'] = df['var_ratio'] = 0
        df['var_compression'] = df['var_expansion'] = 0
    
    # Critical Slowing Down (Insight #2)
    if len(df) >= 63:
        def calc_ac1(x):
            if len(x) < 10:
                return 0
            try:
                return np.corrcoef(x[:-1], x[1:])[0, 1]
            except:
                return 0
        df['ac1'] = df['lagged_ret'].rolling(63, min_periods=10).apply(calc_ac1, raw=True).fillna(0)
        df['ac1_ma21'] = df['ac1'].rolling(21, min_periods=1).mean()
        df['ac1_rising'] = (df['ac1'] > df['ac1_ma21']).astype(float)
        df['ac1_high'] = (df['ac1'] > 0.5).astype(float)
    else:
        df['ac1'] = df['ac1_ma21'] = df['ac1_rising'] = df['ac1_high'] = 0
    
    # Anomalous Dimension (Insight #3)
    if len(df) >= 100:
        deltas = []
        ret_vals = df['lagged_ret'].values
        for i in range(len(df)):
            if i < 50:
                deltas.append(0)
            else:
                window = ret_vals[max(0, i-100):i]
                delta = compute_multiscale_delta(window)
                deltas.append(delta)
        df['anomalous_dimension'] = deltas
        df['delta_ma10'] = pd.Series(deltas).rolling(10, min_periods=1).mean().values
        df['delta_trend'] = pd.Series(deltas).rolling(20, min_periods=1).apply(
            lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) > 1 else 0
        ).values
        df['delta_rising'] = (df['delta_trend'] > 0.05).astype(float)
    else:
        df['anomalous_dimension'] = df['delta_ma10'] = df['delta_trend'] = df['delta_rising'] = 0
    
    # === MARKET TEMPERATURE & COHERENCE ===
    v_cols = [c for c in df.columns if c.startswith('V') and c[1:].isdigit()]
    m_cols = [c for c in df.columns if c.startswith('M') and c[1:].isdigit()]
    s_cols = [c for c in df.columns if c.startswith('S') and c[1:].isdigit()]
    e_cols = [c for c in df.columns if c.startswith('E') and c[1:].isdigit()]
    i_cols = [c for c in df.columns if c.startswith('I') and c[1:].isdigit()]
    p_cols = [c for c in df.columns if c.startswith('P') and c[1:].isdigit()]
    
    # Temperature (Insight #9)
    if v_cols:
        df['v_mean'] = df[v_cols].mean(axis=1)
        df['v_std'] = df[v_cols].std(axis=1)
        df['temperature'] = df['v_std'] / (df['v_mean'].abs() + 1e-8)
        df['temp_ma21'] = df['temperature'].rolling(21, min_periods=1).mean()
        df['temp_high'] = (df['temperature'] > df['temp_ma21'] * 1.5).astype(float)
    else:
        df['v_mean'] = df['v_std'] = df['temperature'] = df['temp_ma21'] = df['temp_high'] = 0
    
    # Order Parameters (Insight #10 - Kuramoto-inspired)
    for prefix, cols in [('V', v_cols), ('M', m_cols), ('S', s_cols)]:
        if len(cols) >= 2:
            zscores = (df[cols] - df[cols].mean()) / (df[cols].std() + 1e-8)
            df[f'{prefix}_order'] = 1 - zscores.var(axis=1).fillna(1)
            df[f'{prefix}_coherence'] = zscores.mean(axis=1).abs()
        else:
            df[f'{prefix}_order'] = df[f'{prefix}_coherence'] = 0
    
    # Volatility Regime
    if v_cols:
        df['vol_regime'] = df[v_cols].mean(axis=1)
        df['vol_regime_ma21'] = df['vol_regime'].rolling(21, min_periods=1).mean()
        df['vol_regime_ma63'] = df['vol_regime'].rolling(63, min_periods=1).mean()
        df['vol_expanding'] = (df['vol_regime'] > df['vol_regime_ma21']).astype(float)
        df['vol_contracting'] = (df['vol_regime'] < df['vol_regime_ma21'] * 0.8).astype(float)
    else:
        df['vol_regime'] = df['vol_regime_ma21'] = df['vol_regime_ma63'] = 0
        df['vol_expanding'] = df['vol_contracting'] = 0
    
    # === FEATURE INTERACTIONS ===
    
    # Sentiment-Vol Interaction (Insight #14)
    if s_cols:
        df['sent_mean'] = df[s_cols].mean(axis=1)
        df['sent_std'] = df[s_cols].std(axis=1)
        df['sent_vol_interact'] = df['sent_mean'] / (df['vol_regime'] + 1e-8)
        df['sent_vol_product'] = df['sent_mean'] * df['vol_regime']
    else:
        df['sent_mean'] = df['sent_std'] = df['sent_vol_interact'] = df['sent_vol_product'] = 0
    
    # Momentum Regime (Insight #16)
    if 'ret_cumsum_21' in df.columns:
        df['cum_ret_std'] = df['ret_cumsum_21'].rolling(63, min_periods=1).std().fillna(0)
        df['momentum_strong'] = (df['ret_cumsum_21'].abs() > df['cum_ret_std']).astype(float)
        df['momentum_direction'] = np.sign(df['ret_cumsum_21'])
    else:
        df['cum_ret_std'] = df['momentum_strong'] = df['momentum_direction'] = 0
    
    # Economic Surprise (Insight #17)
    if e_cols:
        df['econ_mean'] = df[e_cols].mean(axis=1)
        df['econ_std'] = df[e_cols].std(axis=1)
        df['econ_momentum'] = df['econ_mean'].diff(5).fillna(0)
        df['econ_momentum_ma'] = df['econ_momentum'].rolling(63, min_periods=1).mean()
        df['econ_surprise'] = df['econ_momentum'] - df['econ_momentum_ma']
    else:
        df['econ_mean'] = df['econ_std'] = df['econ_momentum'] = df['econ_momentum_ma'] = df['econ_surprise'] = 0
    
    # Interest Rate Regime (Insight #18)
    if len(i_cols) >= 3 and 'I3' in df.columns and 'I1' in df.columns:
        df['rate_slope'] = df['I3'] - df['I1']
        df['rate_slope_ma21'] = df['rate_slope'].rolling(21, min_periods=1).mean()
        df['rate_slope_pct'] = df['rate_slope'].rolling(63, min_periods=1).rank(pct=True).fillna(0.5)
        df['rate_inverting'] = (df['rate_slope_pct'] < 0.1).astype(float)
        df['rate_steepening'] = (df['rate_slope_pct'] > 0.9).astype(float)
    else:
        df['rate_slope'] = df['rate_slope_ma21'] = 0
        df['rate_slope_pct'] = 0.5
        df['rate_inverting'] = df['rate_steepening'] = 0
    
    # === CROSS-DOMAIN CORRELATION (Insight #12) ===
    if v_cols and m_cols and s_cols and len(df) >= 21:
        vm = df[v_cols].mean(axis=1)
        mm = df[m_cols].mean(axis=1)
        sm = df[s_cols].mean(axis=1)
        
        corr_vm = vm.rolling(21, min_periods=5).corr(mm).fillna(0)
        corr_vs = vm.rolling(21, min_periods=5).corr(sm).fillna(0)
        corr_ms = mm.rolling(21, min_periods=5).corr(sm).fillna(0)
        
        df['corr_vm'] = corr_vm
        df['corr_vs'] = corr_vs
        df['corr_ms'] = corr_ms
        
        avg_corr = (abs(corr_vm) + abs(corr_vs) + abs(corr_ms)) / 3
        df['cross_domain_corr'] = avg_corr
        df['correlation_surge'] = (avg_corr > 0.7).astype(float)
        df['correlation_collapse'] = (avg_corr < 0.2).astype(float)
    else:
        df['corr_vm'] = df['corr_vs'] = df['corr_ms'] = 0
        df['cross_domain_corr'] = 0
        df['correlation_surge'] = df['correlation_collapse'] = 0
    
    # === RISK SIGNAL COMPOSITE ===
    df['risk_signal'] = (
        df['var_compression'] * 0.3 +
        df['ac1_rising'] * 0.2 +
        df['delta_rising'] * 0.2 +
        df['correlation_surge'] * 0.2 +
        df['vol_expanding'] * 0.1
    )
    
    return df

print("Full PROMETHEUS feature engineering defined.")

In [ ]:
# Load and process training data
print("Loading training data...")
train_df = pd.read_csv(DATA_DIR / 'train.csv')
print(f"Train shape: {train_df.shape}")

print("\nApplying PROMETHEUS feature engineering...")
df = train_df.copy()
df = df.sort_values('date_id').reset_index(drop=True)
df = add_full_prometheus_features(df)

# Get feature columns
exclude_cols = ['date_id', 'forward_returns', 'risk_free_rate', 
                'market_forward_excess_returns', 'is_scored',
                'lagged_forward_returns', 'lagged_risk_free_rate',
                'lagged_market_forward_excess_returns', 'lagged_ret']

feature_cols = [c for c in df.columns if c not in exclude_cols 
                and df[c].dtype in ['float64', 'int64', 'float32', 'int32']]

print(f"\nTotal features: {len(feature_cols)}")
print(f"\nPROMETHEUS features:")
prometheus_feats = [c for c in feature_cols if any(x in c for x in 
    ['var_', 'ac1', 'delta', 'temperature', 'order', 'coherence', 'vol_regime', 
     'sent_', 'momentum_', 'econ_', 'rate_', 'corr_', 'cross_domain', 
     'correlation_', 'sharpe_', 'risk_signal', '_z'])]
print(f"  Count: {len(prometheus_feats)}")

In [ ]:
# Prepare data for training
df_valid = df[df['market_forward_excess_returns'].notna()].copy()
print(f"Valid samples: {len(df_valid)}")

X = df_valid[feature_cols].fillna(0)
y = df_valid['market_forward_excess_returns']

# Scale features
scaler = RobustScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_cols, index=X.index)

# Time series split
val_size = int(len(X_scaled) * 0.2)
X_train = X_scaled.iloc[:-val_size]
y_train = y.iloc[:-val_size]
X_val = X_scaled.iloc[-val_size:]
y_val = y.iloc[-val_size:]

print(f"Train: {len(X_train)}, Val: {len(X_val)}")
print(f"Features: {len(feature_cols)}")

In [ ]:
# Train ensemble models
print("Training V5 FULL PROMETHEUS ensemble...")

lgb_models = []
xgb_models = []

for seed in range(CONFIG['n_models']):
    print(f"\nModel {seed+1}/{CONFIG['n_models']}...")
    
    # LightGBM
    lgb_params = CONFIG['lgb_params'].copy()
    lgb_params['seed'] = 42 + seed
    
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)
    
    lgb_model = lgb.train(
        lgb_params,
        train_data,
        num_boost_round=CONFIG['num_boost_round'],
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(CONFIG['early_stopping'], verbose=False)]
    )
    lgb_models.append(lgb_model)
    print(f"  LGB iter: {lgb_model.best_iteration}")
    
    # XGBoost
    xgb_params = CONFIG['xgb_params'].copy()
    xgb_params['seed'] = 42 + seed
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    
    xgb_model = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=CONFIG['num_boost_round'],
        evals=[(dval, 'val')],
        early_stopping_rounds=CONFIG['early_stopping'],
        verbose_eval=False
    )
    xgb_models.append(xgb_model)
    print(f"  XGB iter: {xgb_model.best_iteration}")

print(f"\nTrained {len(lgb_models)} LGB + {len(xgb_models)} XGB models.")

In [ ]:
# Validate with FULL PROMETHEUS inference
print("\nValidating with FULL PROMETHEUS inference...")

# Get predictions from all models
all_predictions = []
reliabilities = []

for model in lgb_models:
    pred = model.predict(X_val)
    all_predictions.append(pred)
    reliabilities.append(0.9)  # LGB slightly more reliable

dval = xgb.DMatrix(X_val)
for model in xgb_models:
    pred = model.predict(dval)
    all_predictions.append(pred)
    reliabilities.append(0.85)  # XGB

all_predictions = np.array(all_predictions)
reliabilities = np.array(reliabilities)

# Apply FULL PROMETHEUS FUSION per sample
final_predictions = []
confidences = []
conflicts = []

for i in range(len(X_val)):
    preds_i = all_predictions[:, i]
    
    # Dempster-Shafer fusion (Insight #5)
    ds_pred, ds_conf, ds_conflict = dempster_shafer_fusion(preds_i, reliabilities)
    
    # Value clustering (Insight #6)
    cluster_pred, cluster_conf = value_clustering(preds_i)
    
    # CIC functional (Insight #17)
    cic_F = compute_cic_functional(preds_i)
    
    # Combined prediction
    final_pred = 0.4 * ds_pred + 0.4 * cluster_pred + 0.2 * np.mean(preds_i)
    
    # Combined confidence
    confidence = 0.4 * ds_conf + 0.4 * cluster_conf + 0.2 * cic_F
    
    final_predictions.append(final_pred)
    confidences.append(confidence)
    conflicts.append(ds_conflict)

final_predictions = np.array(final_predictions)
confidences = np.array(confidences)
conflicts = np.array(conflicts)

print(f"Average confidence: {confidences.mean():.3f}")
print(f"Average conflict: {np.mean(conflicts):.3f}")

In [ ]:
# FULL PROMETHEUS POSITION SIZING with risk signals
print("\nApplying FULL PROMETHEUS position sizing...")

cfg = CONFIG
positions = []

# Get validation data features for risk signals
val_df = df_valid.iloc[-val_size:]

for i in range(len(final_predictions)):
    pred = final_predictions[i]
    conf = confidences[i]
    conflict = conflicts[i]
    
    # Get risk signals from features
    row = val_df.iloc[i]
    var_compression = row.get('var_compression', 0)
    ac1_rising = row.get('ac1_rising', 0)
    delta_rising = row.get('delta_rising', 0)
    correlation_surge = row.get('correlation_surge', 0)
    vol_expanding = row.get('vol_expanding', 0)
    
    # Base Kelly position
    std_pred = np.std(all_predictions[:, i])
    uncertainty = max(std_pred, 1e-5)
    kelly = pred / (cfg['risk_aversion'] * uncertainty**2 + 1e-8)
    base_pos = cfg['base_position'] + cfg['scale_factor'] * kelly
    
    # Apply risk adjustments
    risk_factor = 1.0
    
    # Variance compression → reduce (Insight #1)
    if var_compression > 0.5:
        risk_factor *= cfg['var_compression_factor']
    
    # AC1 rising → reduce (Insight #2)
    if ac1_rising > 0.5:
        risk_factor *= cfg['ac1_rising_factor']
    
    # Delta rising → crash risk → reduce aggressively (Insight #3)
    if delta_rising > 0.5:
        risk_factor *= 0.5  # Aggressive reduction
    
    # Correlation surge → reduce (Insight #12)
    if correlation_surge > 0.5:
        risk_factor *= cfg['correlation_surge_factor']
    
    # Vol expanding → reduce (Insight #13)
    if vol_expanding > 0.5:
        risk_factor *= cfg['vol_expanding_factor']
    
    # High conflict → reduce (Insight #5)
    if conflict > 0.5:
        risk_factor *= cfg['high_conflict_factor']
    
    # Apply confidence scaling
    risk_factor *= conf
    
    # Final position
    position = cfg['base_position'] + (base_pos - cfg['base_position']) * risk_factor
    position = np.clip(position, cfg['min_position'], cfg['max_position'])
    positions.append(position)

positions = np.array(positions)
print(f"Position Mean: {positions.mean():.3f}")
print(f"Position Std: {positions.std():.3f}")
print(f"Position Range: [{positions.min():.3f}, {positions.max():.3f}]")

In [ ]:
# Calculate validation metrics
strategy_returns = positions * y_val.values
market_returns = y_val.values

strategy_mean = strategy_returns.mean() * 252
strategy_vol = strategy_returns.std() * np.sqrt(252)
market_vol = market_returns.std() * np.sqrt(252)
sharpe = strategy_mean / (strategy_vol + 1e-8)

# Calculate max drawdown
cumulative = np.cumprod(1 + strategy_returns) - 1
running_max = np.maximum.accumulate(cumulative + 1)
drawdown = (cumulative + 1) / running_max - 1
max_dd = drawdown.min()

# Volatility-adjusted Sharpe (competition metric)
vol_penalty = max(1.0, (strategy_vol / market_vol) / 1.2) if market_vol > 0 else 1.0
adjusted_sharpe = sharpe / vol_penalty

print("\n" + "="*50)
print("V5 FULL PROMETHEUS VALIDATION RESULTS")
print("="*50)
print(f"Strategy Annual Return: {strategy_mean:.4f}")
print(f"Strategy Annual Vol: {strategy_vol:.4f}")
print(f"Market Annual Vol: {market_vol:.4f}")
print(f"Sharpe Ratio: {sharpe:.4f}")
print(f"Adjusted Sharpe: {adjusted_sharpe:.4f}")
print(f"Max Drawdown: {max_dd:.4f}")
print("="*50)

In [ ]:
# Save V5 artifacts
print(f"\nSaving V5 artifacts to {ARTIFACTS_DIR}...")

with open(ARTIFACTS_DIR / 'scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open(ARTIFACTS_DIR / 'feature_cols.pkl', 'wb') as f:
    pickle.dump(feature_cols, f)

with open(ARTIFACTS_DIR / 'lgb_models.pkl', 'wb') as f:
    pickle.dump(lgb_models, f)

for i, model in enumerate(xgb_models):
    model.save_model(str(ARTIFACTS_DIR / f'xgb_model_{i}.json'))

config_to_save = {
    'base_position': cfg['base_position'],
    'risk_aversion': cfg['risk_aversion'],
    'scale_factor': cfg['scale_factor'],
    'min_position': cfg['min_position'],
    'max_position': cfg['max_position'],
    'var_compression_factor': cfg['var_compression_factor'],
    'ac1_rising_factor': cfg['ac1_rising_factor'],
    'correlation_surge_factor': cfg['correlation_surge_factor'],
    'high_conflict_factor': cfg['high_conflict_factor'],
    'vol_expanding_factor': cfg['vol_expanding_factor']
}
with open(ARTIFACTS_DIR / 'config.pkl', 'wb') as f:
    pickle.dump(config_to_save, f)

recent_data = df_valid.tail(300).copy()
recent_data.to_parquet(ARTIFACTS_DIR / 'recent_data.parquet')

print("\nV5 FULL PROMETHEUS artifacts saved!")
print(f"Feature count: {len(feature_cols)}")
print(f"Models: {len(lgb_models)} LGB + {len(xgb_models)} XGB")